In this app, we let the estimated GDP-CO2 relationship vary over time by including time as an additional input into the neural network. 

In [63]:
%matplotlib inline

import matplotlib
import pandas as pd
import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import seaborn as sns; sns.set()

from ipywidgets import interact, fixed
import ipywidgets as widgets

In [64]:
# Ignore UserWarning due to visual tearing in surface plot
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [109]:
# Creating plotting function
def SurfPlot(region, loss, architecture):
    """
    Makes 3D surface plot for a given region in a given year.

    ARGUMENTS
        * region:       Must be from the list: 'World', 'OECD', 'Asia', 'REF', 'MAF', 'LAM'.
        * loss:         Must be from the list: 'MSE', 'MAE'
        * architecture: Must be from the list: (4), (8), (16), (8,4), (16,8,4) 
    """
       
    # Loading data
    time        = np.load('Models/{}/Misc/time.npy'.format(region), allow_pickle=True)
    T           = np.load('Models/{}/Misc/T.npy'.format(region), allow_pickle=True)
    Min         = np.load('Models/{}/Misc/min.npy'.format(region), allow_pickle=True)
    Max         = np.load('Models/{}/Misc/max.npy'.format(region), allow_pickle=True)
    min_vec     = np.load('Models/{}/Misc/min_vec.npy'.format(region), allow_pickle=True)
    max_vec     = np.load('Models/{}/Misc/max_vec.npy'.format(region), allow_pickle=True)
    quantL_vec  = np.load('Models/{}/Misc/quant025_vec.npy'.format(region), allow_pickle=True)
    quantU_vec  = np.load('Models/{}/Misc/quant975_vec.npy'.format(region), allow_pickle=True)
    individuals = np.load('Models/{}/Misc/individuals.npy'.format(region), allow_pickle=True)
    alphas      = np.load('Models/{}/{}/{}/alphas.npy'.format(region, loss, architecture), allow_pickle=True)
    y_train_agg = np.load('Models/{}/Misc/y_train_agg.npy'.format(region), allow_pickle=True)
    
    #lGDP = pd.read_pickle('Models/{}/Misc/lGDP'.format(region))[individuals]
    #lCO2 = pd.read_pickle('Models/{}/Misc/lCO2'.format(region))[individuals]
    
    x_mat_red = np.load('Models/{}/Misc/x_mat_red.npy'.format(region), allow_pickle=True)
    y_mat_red = np.load('Models/{}/Misc/y_mat_red.npy'.format(region), allow_pickle=True)
    
    in_sample_agg = pd.read_pickle('Models/{}/{}/{}/in_sample_agg'.format(region, loss, architecture))
    model_pred    = tf.keras.models.load_model('Models/{}/{}/{}/model_pred'.format(region, loss, architecture))

    # Surface plot
    if region == 'REF':
        ax1 = time - 1989
    else:
        ax1 = time - 1959
    
    ax2      = np.linspace(Min, Max, 1000)

    ax1, ax2 = np.meshgrid(ax1, ax2)

    ax1_vec  = np.reshape(ax1, (-1,1), order='F')
    ax2_vec  = np.reshape(ax2, (-1,1), order='F')
    
    x_input  = np.hstack((ax1_vec, ax2_vec))

    pred     = model_pred(x_input)
    pred     = np.reshape(np.array(pred), (1000, T), order='F')
       
    vec_min = np.reshape(min_vec,(1,-1),order='F')
    vec_max = np.reshape(max_vec,(1,-1),order='F')
    vec_quantL = np.reshape(quantL_vec,(1,-1),order='F')
    vec_quantU = np.reshape(quantU_vec,(1,-1),order='F')
        
    pred_grey = pred.copy()
    pred_grey[np.where(ax2 < vec_min)] = np.nan
    pred_grey[np.where(ax2 > vec_max)] = np.nan
    
    pred_black = pred.copy()
    pred_black[np.where(ax2 < vec_quantL)] = np.nan
    pred_black[np.where(ax2 > vec_quantU)] = np.nan
    
    ax3 = pred.copy()
    ax3[:,:] = 6
    ax3[np.where(ax2 < vec_quantL)] = 5
    #ax3[np.where(ax2 < vec_min)]    = 4
    ax3[np.where(ax2 > vec_quantU)] = 5
    #ax3[np.where(ax2 > vec_max)]    = 4
    
    norm = matplotlib.colors.Normalize(0, 6)
    m = plt.cm.ScalarMappable(norm=norm, cmap='binary')
    m.set_array([])
    fcolors = m.to_rgba(ax3)

    # Scatter plot
    #ax11 = np.hstack([np.reshape(time, (-1,1), order='F')] * lGDP.shape[1])
    #ax22 = np.array(lGDP.loc[time])
    #ax33 = np.array(lCO2.loc[time] - np.hstack((np.zeros((1, 1)), alphas)))
    ax11 = np.hstack([np.reshape(time, (-1,1), order='F')] * x_mat_red.shape[1])
    ax22 = x_mat_red
    ax33 = y_mat_red - np.hstack((np.zeros((1, 1)), alphas))

    # Figure
    plt.close(None)
    
    fig = plt.figure(figsize=(14, 11))

    ax = fig.add_subplot(2, 2, 1, projection='3d')
    #surf = ax.plot_surface(ax1 + 1959, ax2, pred, linewidth=0, antialiased=True, facecolors=fcolors, shade=True)
    surf = ax.plot_surface(ax1 + 1959, ax2, pred_grey, linewidth=0, antialiased=True, facecolors=fcolors, shade=True)
    ax.set_xlabel('year', fontweight='bold')
    ax.set_ylabel('log(GDP)', fontweight='bold')
    ax.set_zlabel('log(CO$_2$)', fontweight='bold')
    #ax.set_ylim(-3,5)
    if region=='World':
        ax.set_zlim(-3,5)
    elif region=='OECD':
        ax.set_zlim(-1,5)
    elif region=='REF':
        ax.set_zlim(-2,5)
    elif region=='Asia':
        ax.set_zlim(-5,7)
    elif region=='MAF':
        ax.set_zlim(-6,7)
    elif region=='LAM':
        ax.set_zlim(-5,6)  
    ax.view_init(36,-140)
    ax.zaxis.labelpad=-0
    plt.title('Estimated surface', fontweight='bold')
    
    ax = fig.add_subplot(2, 2, 2)
    axis = np.reshape(np.array(time), (-1,1), order='F')
        
    ax.plot(axis, in_sample_agg, color='black', label='Model predictions')
    ax.plot(axis, y_train_agg, color='black', linestyle='dashdot', label='Historical emissions')
    ax.set_ylabel('Mt CO$_2$', fontweight='bold')
    ax.set_xlabel('year', fontweight='bold')
    ax.yaxis.labelpad=-2
    if region=='World':
        ax.set_ylim(2000,45000)
    elif region=='OECD':
        ax.set_ylim(2000,15000)
    elif region=='REF':
        ax.set_ylim(0,4500)
    elif region=='Asia':
        ax.set_ylim(0,25000)
    elif region=='MAF':
        ax.set_ylim(0,4000)
    elif region=='LAM':
        ax.set_ylim(0,2000)    
    plt.title('Aggregate predictions', fontweight='bold')
    plt.legend(loc='upper center', bbox_to_anchor=(1.2, 1.02), fancybox=True, shadow=True, ncol=1)
    
    #pred_time = np.mean(pred,axis=1)
    pred_time_black = np.nanmean(pred_black,axis=1)
    pred_time_grey  = np.nanmean(pred_grey,axis=1)
    
    ax = fig.add_subplot(2, 2, 3)
    #plot = ax.plot(np.linspace(Min, Max, 1000), pred_time, antialiased=True, color='black')
    plot1 = ax.plot(np.linspace(Min, Max, 1000), pred_time_grey, antialiased=True, color='dimgrey')
    plot2 = ax.plot(np.linspace(Min, Max, 1000), pred_time_black, antialiased=True, color='black')
    scat = ax.scatter(ax22, ax33, s=2, label='log(CO$_2$) minus estimated country fixed effects')
    ax.set_xlabel('log(GDP)', fontweight='bold')
    ax.set_ylabel('log(CO$_2$)', fontweight='bold')
    ax.yaxis.labelpad=-2
    #ax.set_ylim(-3,5)
    if region=='World':
        ax.set_ylim(-4,5)
    elif region=='OECD':
        ax.set_ylim(-1,5)
    elif region=='REF':
        ax.set_ylim(-2,5)
    elif region=='Asia':
        ax.set_ylim(-5,7)
    elif region=='MAF':
        ax.set_ylim(-6,7)
    elif region=='LAM':
        ax.set_ylim(-5,6)  
    plt.title('Average of estimated surface across time-dimension', fontweight='bold')
    #plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.14), fancybox=True, shadow=True, ncol=1)
    
    
    #pred_GDP = np.mean(pred,axis=0)
    pred_GDP_black = np.nanmean(pred_black,axis=0)
    pred_GDP_grey  = np.nanmean(pred_grey,axis=0)

    ax = fig.add_subplot(2, 2, 4)
    #plot1 = ax.plot(np.array(time), pred_GDP, antialiased=True, color='black')
    plot1 = ax.plot(np.array(time), pred_GDP_grey, antialiased=True, color='dimgrey', label='Grey surface')
    plot2 = ax.plot(np.array(time), pred_GDP_black, antialiased=True, color='black', label='Black surface')
    #plot2 = ax.plot(np.array(time), np.nanmean(ax33, axis=1), antialiased=True, color='black', linestyle='dashed')
    scat = ax.scatter(ax11, ax33, s=2)
    ax.set_xlabel('year', fontweight='bold')
    ax.set_ylabel('log(CO$_2$)', fontweight='bold')
    ax.yaxis.labelpad=-2
    #ax.set_ylim(-3,5)
    if region=='World':
        ax.set_ylim(-4,5)
    elif region=='OECD':
        ax.set_ylim(-1,5)
    elif region=='REF':
        ax.set_ylim(-2,5)
    elif region=='Asia':
        ax.set_ylim(-5,7)
    elif region=='MAF':
        ax.set_ylim(-6,7)
    elif region=='LAM':
        ax.set_ylim(-5,6)  
    plt.title('Average of estimated surface across GDP-dimension', fontweight='bold')
    plt.legend(loc='upper center', bbox_to_anchor=(1.155, 1.02), fancybox=True, shadow=True, ncol=1)
    
    plt.show()
    

In [110]:
interact(SurfPlot, region=['World', 'OECD', 'REF', 'Asia', 'MAF', 'LAM'], loss=['MSE', 'MAE'], architecture=['(4)', '(8)', '(16)', '(8,4)', '(16,8,4)']);

interactive(children=(Dropdown(description='region', options=('World', 'OECD', 'REF', 'Asia', 'MAF', 'LAM'), v…

Notes: <br>
Grey surface marks the region between the yearly min and max log-GDP. <br>
Black surface marks the region between the yearly .025 and .975 log-GDP quantile. <br>
Blue dots represent log-CO$_2$ in a given country in a given year minus its estimated country fixed effect.